<a href="https://colab.research.google.com/github/IvlevaAnna1/Hometasks/blob/main/ML_2_HW_IvlevaAnna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [8]:
y = data['MEDV']
X = data.drop(columns=['MEDV'])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10)
X_train.shape, X_test.shape, y_train.shape, y_test.shape
### Ваш код ###

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [36]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)
### Ваш код ###
rid = Ridge()
rid.fit(X_train, y_train)
y_pred_rid = rid.predict(X_test)
r2_ridge = r2_score(y_test, y_pred_rid)
las = Lasso()
las.fit(X_train, y_train)
y_pred_las = las.predict(X_test)
r2_lasso = r2_score(y_test, y_pred_las)
r2_lr = 0.6706
r2_ridge = 0.6652
r2_lasso = 0.5346
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]
print(r2_lr)
print(r2_ridge)
print(r2_lasso)


0.6706
0.6652
0.5346


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

Алексей, добрый день! Просьба уточнить, по каким параметрам подбирать коэффициент способом GridSearchCV.

In [54]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
# укажем параметры регуляризации alpha, которые хотим протестировать
# дополнительно укажем количество частей (folds), параметр cv,
# на которое нужно разбить данные при оценке качества модели
ridge_cv = RidgeCV(alphas = [10**(-5), 10**(-4),10**(-3),10**(-2),10**(-1),1,10,10**(2),10**(3),10**(4),10**(5)], cv = 10)
ridge_cv.fit(X_train, y_train)
lasso_cv = LassoCV(alphas = [10**(-5), 10**(-4),10**(-3),10**(-2),10**(-1),1,10,10**(2),10**(3),10**(4),10**(5)], cv = 10)
lasso_cv.fit(X_train, y_train)

param_grid = {'alpha': [10**(-5), 10**(-4),10**(-3),10**(-2),10**(-1),1,10,10**(2),10**(3),10**(4),10**(5)]}
rid = Ridge()
grid = GridSearchCV(estimator=rid, param_grid=param_grid, scoring='r2')
grid.fit(X_train, y_train)
print(grid.best_params_)

param_grid2 = {'alpha': [10**(-5), 10**(-4),10**(-3),10**(-2),10**(-1),1,10,10**(2),10**(3),10**(4),10**(5)]}
las = Lasso()
grid2 = GridSearchCV(estimator=las, param_grid=param_grid2, scoring='r2')
grid2.fit(X_train, y_train)
print(grid2.best_params_)

# выведем оптимальный параметр и достигнутое качество
ridge_cv.alpha_, ridge_cv.best_score_
lasso_cv.alpha_
print(ridge_cv.alpha_)
print(lasso_cv.alpha_)


rid2 = Ridge(alpha = 0.1)
rid2.fit(X_train, y_train)
y_pred_rid2 = rid2.predict(X_test)
r2_ridge_cv = r2_score(y_test, y_pred_rid2)

las2 = Lasso(alpha = 10**(-5))
las2.fit(X_train, y_train)
y_pred_las2 = las2.predict(X_test)
r2_lasso_cv = r2_score(y_test, y_pred_las2)

las3 = Lasso(alpha = 0.001)
las3.fit(X_train, y_train)
y_pred_las3 = las3.predict(X_test)
r2_lasso_grid_search = r2_score(y_test, y_pred_las3)

print(r2_ridge_cv)
print(r2_lasso_cv)
print(r2_lasso_grid_search)
### Ваш код ###

r2_ridge_grid_search = 0.6700
r2_ridge_cv = 0.6700
r2_lasso_grid_search = 0.6703
r2_lasso_cv = 0.6706
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

{'alpha': 0.1}
{'alpha': 0.001}
0.1
1e-05
0.6700015276142122
0.6706391690710081
0.6703600853352256


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, StandardScaler

# Создание Pipeline
pipe_ridge_standart_scaler = Pipeline([
    ('scaler', StandardScaler()),  # Этап масштабирования данных
    ('regressor', Ridge())  # Этап обучения модели линейной регрессии
])
pipe_ridge_min_max_scaler = Pipeline([
    ('scaler', MinMaxScaler()),  # Этап масштабирования данных
    ('regressor', Ridge())  # Этап обучения модели линейной регрессии
])
pipe_lasso_standart_scaler = Pipeline([
    ('scaler', StandardScaler()),  # Этап масштабирования данных
    ('regressor', Lasso())  # Этап обучения модели линейной регрессии
])
pipe_lasso_min_max_scaler = Pipeline([
    ('scaler', MinMaxScaler()),  # Этап масштабирования данных
    ('regressor', Lasso())  # Этап обучения модели линейной регрессии
])

# Обучение модели
pipe_ridge_standart_scaler.fit(X_train, y_train)
pipe_ridge_min_max_scaler.fit(X_train, y_train)
pipe_lasso_standart_scaler.fit(X_train, y_train)
pipe_lasso_min_max_scaler.fit(X_train, y_train)

# Предсказание на тестовом наборе данных
y_pred_ridge_standart_scaler = pipe_ridge_standart_scaler.predict(X_test)
y_pred_ridge_min_max_scaler = pipe_ridge_min_max_scaler.predict(X_test)
y_pred_lasso_standart_scaler = pipe_lasso_standart_scaler.predict(X_test)
y_pred_lasso_min_max_scale = pipe_lasso_min_max_scaler.predict(X_test)

# Расчет ошибки
r2_ridge_standart_scaler = r2_score(y_test, y_pred_ridge_standart_scaler)
r2_ridge_min_max_scaler = r2_score(y_test, y_pred_ridge_min_max_scaler)
r2_lasso_standart_scaler = r2_score(y_test, y_pred_lasso_standart_scaler)
r2_lasso_min_max_scaler = r2_score(y_test, y_pred_lasso_min_max_scale)
### Ваш код ###
print(r2_ridge_standart_scaler)
print(r2_ridge_min_max_scaler)
print(r2_lasso_standart_scaler)
print(r2_lasso_min_max_scaler)


r2_ridge_standart_scaler = 0.6708
r2_ridge_min_max_scaler = 0.6662
r2_lasso_standart_scaler = 0.6081
r2_lasso_min_max_scaler = 0.0943
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

0.6708439611875984
0.6662592464569836
0.6081498113418891
0.09431861897045679


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:
standart_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()



### Ваш код ###

r2_ridge_standart_scaler_cv = 0
r2_ridge_min_max_scaler_cv = 0
r2_lasso_standart_scaler_cv = 0
r2_lasso_min_max_scaler_cv = 0
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [ ]:


### Ваш код ###

r2_ridge_standart_scaler_poly = 0
r2_ridge_min_max_scaler_poly = 0
r2_lasso_standart_scaler_poly = 0
r2_lasso_min_max_scaler_poly = 0
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:


### Ваш код ###

r2_ridge_standart_scaler_poly_cv = 0
r2_ridge_min_max_scaler_poly_cv = 0
r2_lasso_standart_scaler_poly_cv = 0
r2_lasso_min_max_scaler_poly_cv = 0
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
r2_best_model = 0
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

In [55]:
results_regression

,model,task,R2
0,LR,task2,0.6706
1,Ridge,task2,0.6652
2,Lasso,task2,0.5346
7,Ridge_StandardScaler,task4,0.6708
8,Ridge_MinMaxScaler,task4,0.6662
9,Lasso_StandardScaler,task4,0.6081
10,Lasso_MinMaxScaler,task4,0.0943
3,Ridge_GridSearchCV,task3,0.6700
4,RidgeCV,task3,0.6700
5,Lasso_GridSearchCV,task3,0.6703


http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
data = pd.read_csv('adult.csv')
data.head()

9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [ ]:


### Ваш код ###

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [ ]:


### Ваш код ###

f1_most_frequent = 0
acc_most_frequent = 0
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [ ]:


### Ваш код ###

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [ ]:


### Ваш код ###

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:


### Ваш код ###

f1_LR_del_missings = 0
acc_LR_del_missings = 0
f1_KNN_del_missings = 0
acc_KNN_del_missings = 0
f1_SVM_del_missings = 0
acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:


### Ваш код ###

f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [ ]:
results_classification